<a href="https://colab.research.google.com/github/BatoolAyman/Deep_Learning/blob/main/CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# 1. Data Loading and Preprocessing
# Define a transform to convert images to tensors.
transform = transforms.Compose([
    transforms.ToTensor(),
    # Optionally, add normalization:
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [4]:
# Download and load the CIFAR-10 training and test datasets.
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)


100%|██████████| 170M/170M [00:02<00:00, 76.4MB/s]


In [5]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

In [6]:
# 2. Define the CNN Model
# Here we subclass nn.Module to define a simple CNN with two convolutional layers,
# pooling, and two fully-connected layers.
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # Convolutional layer 1: from 3 input channels to 32 output channels
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        # Activation (ReLU) and pooling follow
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Convolutional layer 2: from 32 channels to 64 channels
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        # A fully connected layer that takes the flattened features.
        # For CIFAR-10 images of size 32x32, after two poolings (each halving dimensions),
        # the spatial size becomes 8x8, and with 64 channels, that gives 64*8*8 = 4096 features.
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)  # 10 output classes

    def forward(self, x):
        # First conv layer + activation + pooling.
        x = self.pool(self.relu(self.conv1(x)))
        # Second conv layer + activation + pooling.
        x = self.pool(self.relu(self.conv2(x)))
        # Flatten the feature map to a vector
        x = x.view(x.size(0), -1)
        # Fully connected layers with activation
        x = self.relu(self.fc1(x))
        # Final fully connected layer (logits output)
        x = self.fc2(x)
        return x

In [7]:
# Instantiate the model
model = SimpleCNN().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        # Get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(trainloader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

print('Finished Training')

Epoch [1/10], Step [100/1563], Loss: 1.9160
Epoch [1/10], Step [200/1563], Loss: 1.5747
Epoch [1/10], Step [300/1563], Loss: 1.4594
Epoch [1/10], Step [400/1563], Loss: 1.3773
Epoch [1/10], Step [500/1563], Loss: 1.3367
Epoch [1/10], Step [600/1563], Loss: 1.2598
Epoch [1/10], Step [700/1563], Loss: 1.2656
Epoch [1/10], Step [800/1563], Loss: 1.1860
Epoch [1/10], Step [900/1563], Loss: 1.1602
Epoch [1/10], Step [1000/1563], Loss: 1.1288
Epoch [1/10], Step [1100/1563], Loss: 1.1082
Epoch [1/10], Step [1200/1563], Loss: 1.0794
Epoch [1/10], Step [1300/1563], Loss: 1.0416
Epoch [1/10], Step [1400/1563], Loss: 1.0341
Epoch [1/10], Step [1500/1563], Loss: 1.0490
Epoch [2/10], Step [100/1563], Loss: 0.9295
Epoch [2/10], Step [200/1563], Loss: 0.8950
Epoch [2/10], Step [300/1563], Loss: 0.9086
Epoch [2/10], Step [400/1563], Loss: 0.8747
Epoch [2/10], Step [500/1563], Loss: 0.8465
Epoch [2/10], Step [600/1563], Loss: 0.8526
Epoch [2/10], Step [700/1563], Loss: 0.8535
Epoch [2/10], Step [800/15

In [9]:
# Evaluating the model
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

Accuracy of the model on the test images: 71.03%
